In [4]:
from scipy.stats import mode
import pandas as pd
import statistics
import datetime
from statistics import mean
import warnings 
warnings.filterwarnings("ignore")

In [5]:
sales = pd.ExcelFile(r'C:\Users\ADMIN\Downloads\Codes\TimeSeries Demand Classification & Forecasting\online_retail_II.xlsx')
sales1 = sales.parse('Year 2009-2010')
sales2 = sales.parse('Year 2010-2011')
sales1 = sales1.append(sales2)
sales = sales1
del sales1
del sales2

In [6]:
sales.replace(',','', regex=True, inplace=True)
sales['Quantity'] = pd.to_numeric(sales['Quantity'])
sales = sales.loc[sales['Quantity']>0]
sales['InvoiceDate'] = pd.to_datetime(sales['InvoiceDate'])
sales['InvoiceDate'] = sales['InvoiceDate'].dt.date

In [7]:
sales = sales.groupby(['StockCode','Description','InvoiceDate']).agg({
                    'Quantity':sum
}).reset_index()

In [8]:
new_columns = ['InvoiceDate','StockCode','Description','Quantity']
sales = sales[new_columns]

In [9]:
imputed_sales = pd.DataFrame()
output = pd.DataFrame()
def data_processing(df):
    df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
    min_date = df['InvoiceDate'].min()
    max_date = df['InvoiceDate'].max()
    all_dates = pd.to_datetime(pd.date_range(start=min_date, end=max_date))
    date_range_df = pd.DataFrame({'InvoiceDate':all_dates})
    df_merge = date_range_df.merge(df, on='InvoiceDate', how='left')
    material_id = df_merge['StockCode'][0]
    df_merge['StockCode'].fillna(material_id, inplace=True)
    df_merge['Quantity'] = df_merge['Quantity'].fillna(0)
    df_merge = df_merge.set_index('InvoiceDate').resample("M").agg({'Quantity': 'sum', 'StockCode': 'last'}).reset_index()
    global imputed_sales
    imputed_sales = imputed_sales.append(df_merge,ignore_index=True)
    adi = len(df_merge)/len(df_merge.loc[df_merge['Quantity']>0])
    if(len(df_merge)>2):
        CV2 = (statistics.stdev(df_merge['Quantity']>0) / statistics.mean(df_merge['Quantity']>0))**2
        if(adi < 1.32):
            if(CV2 < 0.49):
                classification = 'Smooth demand'
            if(CV2 >= 0.49):
                classification = 'Erratic demand'
        elif(adi >= 1.32):
            if(CV2 < 0.49):
                classification = 'Intermittent demand'
            if(CV2 >= 0.49):
                classification = 'Lumpy demand'
    else:
        CV2 = 'NA'
        classification = 'NA'
    material_desc = mode(df['Description'])[0][0]
    temp_dict = {'StockCode':material_id,'Description':material_desc,'ADI':adi,'CV2':CV2,'Class': classification }
    global output
    output = output.append(temp_dict,ignore_index=True)
    pass

In [10]:
sales.groupby(['StockCode']).apply(data_processing)

""


In [11]:
output.to_csv(r"C:\Users\ADMIN\Downloads\Codes\TimeSeries Demand Classification & Forecasting\Demand Classification\Monthly Frequency\Forecast Classification Daily.csv")
imputed_sales.to_csv(r"C:\Users\ADMIN\Downloads\Codes\TimeSeries Demand Classification & Forecasting\Demand Classification\Monthly Frequency\Imputed Sales Daily.csv")